In [78]:
import json
import os

import numpy as np
from datetime import datetime,date
from utils.gcp import GoogleSheetsClient
import pandas as pd
import json
import numpy as np

from google.cloud import bigquery
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/Users/marianonieto/Desktop/challenge/Challenge/secret.json'

with open(os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')) as f:
   secret = json.load(f)

In [79]:
client = bigquery.Client()
cliente_sheet=GoogleSheetsClient(secret)

In [ ]:
def execute_response(client,query):
    rows=client.query(query).result()
    df=[dict(row) for row in rows]
    return df

### Limpio csv 

#### Loan Data

In [52]:
df=pd.read_csv('loan_data.csv')

In [53]:
df.sample(5)

,customer_id,loan_id,application_date,term,disbursed_date,loanamount,paid_date,is_disbursed
3873,5y4204g78g6yh5h24g9788zz,loan_7641841,24/03/22 18:18,6.0,24/03/22,1750.540,NaN,True
429,620g9i6810xi62i450y0141i,loan_2951846,16/02/22 19:49,NaN,NaN,1459.270,NaN,False
3658,61zx51573z4gizi9254yhxih,loan_2950183,04/01/22 10:01,NaN,NaN,6569.270,NaN,False
3354,60i49ghgg160z0i56g871144,loan_7667104,29/11/21 14:30,NaN,NaN,2771.810,NaN,False
637,61z021001088yg21z299443y,loan_4814599,30/03/22 13:45,8.0,30/03/22,1136.464,NaN,True


In [54]:
#df.replace({np.nan: None}, inplace = True)
df['application_date']=pd.to_datetime(df['application_date'], format='%d/%m/%y %H:%M')#.dt.tz_localize('America/Mexico_City')
#df['application_date_2']=pd.to_datetime(df['application_date'], format='%d/%m/%y %H:%M')
df['disbursed_date']=pd.to_datetime(df['disbursed_date'], format='%d/%m/%y').dt.strftime("%Y/%m/%d")
df['paid_date']=pd.to_datetime(df['paid_date'], format='%d/%m/%y').dt.strftime("%Y/%m/%d")

df.replace({np.nan: None}, inplace = True)


In [55]:
df.sample(5)

,customer_id,loan_id,application_date,term,disbursed_date,loanamount,paid_date,is_disbursed
11868,619y41999zxz088yix5997z9,loan_4314339,2021-11-24 16:05:00,4.0,2021/11/24,5313.9255,2022/01/29,True
6669,619g7yz8860iiy791x14i342,loan_4708742,2022-01-27 16:02:00,None,None,9343.27,None,False
2615,5g96x6iigz030z729yx3hy95,loan_1238551,2022-04-13 22:24:00,8.0,2022/04/13,1605.27,None,True
1906,621i1y5yxhzz379i177568xh,loan_7330329,2022-03-29 18:59:00,None,None,6345.89,None,False
8391,61944y3iih59xg26zy21yih2,loan_6254910,2022-04-23 12:59:00,None,None,1923.55,None,False


In [56]:
df.to_csv('loan_data_ok.csv',index=False)

In [57]:
#folder_id='1GO-vprPsG5kdCcj5HpbZB0v-lTFv92Le'
sheets_id='10v9266PP-kRDvJLucrrhdGv5U1NWrzFr_gC9ASz747c'

In [58]:
df=pd.read_csv('loan_data_ok.csv')

In [59]:
df.replace({np.nan: None}, inplace = True)

In [60]:
df.sample(10)

,customer_id,loan_id,application_date,term,disbursed_date,loanamount,paid_date,is_disbursed
11838,61hx0xy83h3zy65341z45g9h,loan_5945865,2022-04-26 22:43:00,8.0,2022/04/26,1896.54,None,True
5762,5h1yxz2h69x6723y815yi293,loan_3807365,2021-11-29 14:46:00,None,None,6305.74,None,False
12772,5x32x39zxz02g0222yz9561i,loan_8884125,2022-03-21 20:08:00,6.0,2022/03/21,2176.13,None,True
9769,60z67h54gy528hi4321954i2,loan_5502588,2021-12-30 21:55:00,None,None,3639.05,None,False
7931,621z8482z227h922gi534665,loan_1174335,2022-02-26 13:55:00,None,None,6788.27,None,False
1562,6192ix8i470ixhz13i44yh94,loan_5430192,2021-11-29 19:40:00,None,None,5109.27,None,False
2119,5y37099xg731076zh22539i7,loan_1604514,2021-12-24 12:09:00,4.0,2021/12/24,557.8952,None,True
4230,618xx677yy496x3479688007,loan_9926977,2022-04-13 06:44:00,None,None,5537.05,None,False
9751,6067h0xg0zgzxhg6462yyx52,loan_1866656,2022-02-18 20:45:00,4.0,2022/02/18,639.2683,2022/04/16,True
8431,62200yy3z3835g847i04i119,loan_0409323,2022-04-15 21:09:00,None,None,2952.8135,None,False


In [62]:
cliente_sheet.delete_columns(sheets_id, 'loan_data', 0,len(df.columns))

{'spreadsheetId': '10v9266PP-kRDvJLucrrhdGv5U1NWrzFr_gC9ASz747c',
 'replies': [{}]}

In [63]:

print(f"Uploading sheet ")
cliente_sheet.upload_dataframe_to_spreadsheet(
    df, sheets_id, tab_name=f"loan_data", start_cell="A1"
)

Uploading sheet 


{'spreadsheetId': '10v9266PP-kRDvJLucrrhdGv5U1NWrzFr_gC9ASz747c',
 'updates': {'spreadsheetId': '10v9266PP-kRDvJLucrrhdGv5U1NWrzFr_gC9ASz747c',
  'updatedRange': 'loan_data!A1:H13168',
  'updatedRows': 13168,
  'updatedColumns': 8,
  'updatedCells': 78422}}

In [70]:
#cliente_sheet.grant_permissions('1BFF-feWGcFhlIe1ZPFmFDdCzbAyR0Z8TIL69y7fiKvU', "writer", 'sofia.fsc.challenge@gmail.com')

#### Repayment data

In [64]:
repayment=pd.read_csv('repayment_data.csv')

In [65]:
repayment.sample(5)

,loan_id,installment_id,installment_number,installment_duedate,repaid_date,capital_due,capital_paid,interests_paid
5220,loan_9631339,8z7804i87i408701017i430zx07i0843,1,2022-03-15,2022-03-14,100.7400,100.7400,0.00
6857,loan_5213688,8z782ii57yy6x22x017yy8h8ygzi1188,2,2021-11-30,2021-11-30,206.7287,206.7287,72.81
15456,loan_8281943,8z780i5i7x34g32y017x3599gzh34g97,3,2022-02-15,NaN,326.7115,207.7215,168.45
18645,loan_8681724,8z780y0x7xg58431017xg94h261g3503,2,2022-02-28,NaN,719.2982,0.0000,0.00
7910,loan_2046468,8z780831801h522801802502001z01i2,1,2022-05-02,2022-05-02,559.1800,559.1800,0.00


In [68]:


repayment['installment_duedate']=pd.to_datetime(repayment['installment_duedate'], format='%Y-%m-%d').dt.strftime("%Y-%m-%d")
repayment['repaid_date']=pd.to_datetime(repayment['repaid_date'], format='%Y-%m-%d').dt.strftime("%Y-%m-%d")

repayment.replace({np.nan: None}, inplace = True)

In [69]:
repayment.sample(5)

,loan_id,installment_id,installment_number,installment_duedate,repaid_date,capital_due,capital_paid,interests_paid
28610,loan_1164986,8z7807z37h5y79h2017h601747x26238,1,2021-12-15,2021-12-12,415.3700,415.37,0.0
11746,loan_9187700,8z7806h27x9gy1yy017x9hy31xh178x3,4,2022-03-31,2022-03-30,230.6800,230.68,0.0
17783,loan_4818189,8z782i8z7ixi704i017ii1g27223060h,4,2022-05-31,None,135.9625,0.00,0.0
24915,loan_5045805,8z78034h7iyg91zz017iyxy39g9628g1,4,2022-05-31,None,870.8900,0.00,0.0
363,loan_0256495,8z782hy97y5ixz5z017y61h1h279765i,7,2022-01-31,2022-01-31,315.3600,315.36,0.0


In [72]:
sheet_id_repayment='1ypPjTFlYmywHATaznm9E8oCrDeSA_cGc7wL3PpDDors'


In [ ]:
cliente_sheet.delete_columns(sheet_id_repayment, 'repayment_data', 0,len(repayment.columns))

In [77]:

print(f"Uploading sheet ")
cliente_sheet.upload_dataframe_to_spreadsheet(
    repayment, sheet_id_repayment, tab_name="repayment_data", start_cell="A1"
)

Uploading sheet 


{'spreadsheetId': '1ypPjTFlYmywHATaznm9E8oCrDeSA_cGc7wL3PpDDors',
 'updates': {'spreadsheetId': '1ypPjTFlYmywHATaznm9E8oCrDeSA_cGc7wL3PpDDors',
  'updatedRange': 'repayment_data!A1:H29437',
  'updatedRows': 29437,
  'updatedColumns': 8,
  'updatedCells': 222840}}

## Generación tablas productivas
Una vez que se realiza la tabla de STG apuntando al google Sheet, se terminan de definir los tipos de datos de las columnas y se genera una tabla productiva

In [ ]:
query_productive_loan_data="""create or replace table PROD.BT_LOAN_DATA as 
                                SELECT customer_id,loan_id,
                                DATETIME(application_date, 'America/Mexico_City') as application_date,
                                cast(term as int) as term,
                                PARSE_DATE ('%Y/%m/%d',disbursed_date) as disbursed_date,
                                cast(loanamount as numeric) as loanamount ,
                                PARSE_DATE ('%Y/%m/%d',paid_date) as paid_date,
                                cast(is_disbursed as bool) as is_disbursed
                                FROM `sofia-fasce.STG.BT_LOAN_DATA`""" 



In [ ]:
query_productive_repayment_data= """create or replace table `sofia-fasce.PROD.BT_REPAYMENT_DATA` as 
                                SELECT 
                                loan_id,	
                                installment_id,
                                cast(installment_number as int) as installment_number,
                                installment_duedate,
                                repaid_date,
                                cast(capital_due as numeric) as capital_due ,
                                cast(capital_paid as numeric) as capital_paid ,
                                cast(interests_paid as numeric) as interests_paid 
                                FROM `sofia-fasce.STG.BT_REPAYMENT_DATA`"""

In [ ]:
query_recurrence="""create table PROD.BT_RECURRENCE AS
with loan_data as (
select a.*,
row_number() over (partition by customer_id order by disbursed_date) recurrence_1,
case when min(paid_date) over (partition by customer_id order by disbursed_date  ROWS UNBOUNDED PRECEDING )<= disbursed_date then 1 else 0 end as paid_any_loan_last
  from `PROD.BT_LOAN_DATA` a
  where 
  
  is_disbursed=true
),
repay_complete_data as (
 select a.customer_id	,
        a.loan_id	,
        a.term,
        a.disbursed_date	,
        a.paid_date	,
        a.recurrence_1	,
        a.paid_any_loan_last	,
        b.installment_number	,
        b.repaid_date	
 from loan_data a
   join `PROD.BT_REPAYMENT_DATA` b on a.loan_id=b.loan_id
)
select customer_id,loan_id,recurrence_1,case when first_loan_3_or_more_terms=1 or paid_any_loan_last=1 then True else False end as recurrence_2   from (
select * ,
case when min(case when recurrence_1=1	and	installment_number>=3 then	repaid_date else null end) over(partition by customer_id order by disbursed_date  ROWS UNBOUNDED PRECEDING ) <=disbursed_date then 1 else 0 end first_loan_3_or_more_terms
from repay_complete_data
) group by 1,2,3,4
"""

### Control de cantidades

### Idea inicial 
Limpiar los datos con python y volcar la información directo a la tabla en BQ, el problema que se presentó es que en la versión trial de BQ no se permite el insert de datos.
La solución fue limpiar en Python, hacer una volcada de datos en google sheet desde python y apuntar una tabla desde BQ al Gsheet.

In [34]:
table=client.get_table('sofia-fasce.STG.BT_LOAN_DATA')

In [ ]:
client.insert_rows_from_dataframe(table=table, dataframe=df )

In [94]:
query="""select * from sofia-fasce.PROD.BT_RECURRENCE"""
rows=client.query(query).result()


In [96]:
df=pd.DataFrame(df_ok)

In [97]:
len(df)

4908

In [98]:
df.to_csv('RECURRENCE.csv',index=False)

In [99]:
df.sample(10)

,customer_id,loan_id,recurrence_1,recurrence_2
1887,6192ggi9zy2y6zh9001z31xg,loan_9783765,1,False
687,623z22y4364209z6zyi3i7yh,loan_8741556,1,False
4169,618y5hz9i02z4ziy65hhxx15,loan_8726866,3,True
2740,59x52yzg1ih2917h0096h3y5,loan_0524735,1,False
3520,6163211891184gz62z0442yg,loan_3629438,2,True
2417,621h535g04257xh5i8g1x575,loan_1954761,1,False
2452,625661264hx3785yxx19g590,loan_0325728,1,False
2486,619h5811g0281z1x3zg53i83,loan_7836170,1,False
1508,619x95530x9y1xg05g45h898,loan_9481881,1,False
808,61i7ih2510h240250852x0gz,loan_7194574,1,False
